# Code generation with `pystella` and `loopy`

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cla
import pyopencl.clrandom as clr
import loopy as lp
from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2

# Four ways to an OpenCL kernel

We're going to create (and run!) an OpenCL kernel that computes

$$
a(\mathbf{x}) = b(\mathbf{x})^2 \cdot c(\mathbf{x}) + z
$$

in four different ways.

First, we'll generate data and expected results with `numpy`.

In [2]:
n = 64  # the grid size in each dimension

b_h = np.random.rand(n, n, n).astype(np.float64)
c_h = np.random.rand(n, n, n).astype(np.float64)
z = np.array(3.2)

a_true_h = b_h**2 * c_h + z

## 1. `pyopencl` arrays methods

First, we need an OpenCL "context" (the umbrella construct for running programs with OpenCL) and a "queue" (to which kernels will be submitted to execute on a device).

Check out `pyopencl`'s [docs](https://documen.tician.de/pyopencl/) for examples and details.

In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

`pyopencl` has a very convenient `Array` construct, which emulates `numpy` arrays---but with memory residing on the device.
We'll copy the data to the device and try it out.

In [4]:
b = cla.to_device(queue, b_h)
c = cla.to_device(queue, c_h)
a = cla.zeros_like(b)
a_true = cla.to_device(queue, a_true_h)

In [5]:
a[:] = b**2 * c + z

To compare results, compute the maximum of `a - a_true`:

In [6]:
difference = a - a_true

In [7]:
np.max(difference.get())

8.881784197001252e-16

Note that we had to call `difference.get()`, which returns a `numpy.ndarray` on the "host" (the CPU) with data copied from `difference` (on the GPU).
We can also use `pyopencl`'s `max` method:

In [8]:
cla.max(difference)

array(8.8817842e-16)

## 2. OpenCL kernel generation with `loopy`

Refer to `loopy`'s [tutorial](https://documen.tician.de/loopy/tutorial.html) to get started.

Let's create a kernel which computes the above for $i \in [0, N_x)$, $j \in [0, N_y)$, and $k \in [0, N_z)$:

In [9]:
knl = lp.make_kernel(
    "{[i, j, k]: 0 <= i < Nx and 0 <= j < Ny and 0 <= k < Nz}",
    """
    a[i, j, k] = b[i, j, k]**2 * c[i, j, k] + z
    """
)

Inspect your kernel to see if it appears correct by printing it.
How did `make_kernel` interpret the un-indexed scalar variable `z`?

In [10]:
print(knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: <auto/runtime>
Ny: ValueArg, type: <auto/runtime>
Nz: ValueArg, type: <auto/runtime>
a: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
b: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
c: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
z: ValueArg, type: <auto/runtime>
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [i, j, k] : 0 <= i < Nx and 0 <= j < Ny and 0 <= k < Nz }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i: None
j: None
k: None
-----------------------------------------

We can now test our kernel by directly calling the `knl` we created above:

In [11]:
evt, _ = knl(queue, a=a, b=b, c=c, z=z)

Note that `z` needs to be a `numpy.array` so that `loopy` can infer its datatype.

To compare results, compute the maximum of `a - a_true`:

In [12]:
difference = a - a_true

In [13]:
np.max(difference.get())

8.881784197001252e-16

### Parallelization

Now, GPUs are parallel, and the kernel we just wrote isn't making use of any parallelism.
First, let's check what OpenCL code was produced by setting the kernel option `write_cl` to `True`:

In [14]:
knl = lp.set_options(knl, write_cl=True)

If we run the kernel now, it will print OpenCL code:

In [15]:
evt, _ = knl(queue, a=a, b=b, c=c, z=z)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(int const Nx, int const Ny, int const Nz, __global double *__restrict__ a, __global double const *__restrict__ b, __global double const *__restrict__ c, double const z)
{
  for (int k = 0; k <= -1 + Nz; ++k)
    if (-1 + Nx >= 0 && -1 + Ny >= 0)
      for (int j = 0; j <= -1 + Ny; ++j)
        for (int i = 0; i <= -1 + Nx; ++i)
          a[Nz * Ny * i + Nz * j + k] = c[Nz * Ny * i + Nz * j + k] * b[Nz * Ny * i + Nz * j + k] * b[Nz * Ny * i + Nz * j + k] + z;
}



It looks correct, array indexing and all.
But that's a lot of sequential loops!
`loopy` enables *code transformations* that (aim to) optimize the performance of a given kernel.
For instance, mapping the `k` index to the "0" index of the local and global OpenCL thread dimensions is accomplished via `loopy.split_iname`:

In [16]:
knl = lp.split_iname(knl, "k", 32, outer_tag="g.0", inner_tag="l.0")

Let's see what this did to the kernel:

In [17]:
print(knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: <auto/runtime>
Ny: ValueArg, type: <auto/runtime>
Nz: ValueArg, type: <auto/runtime>
a: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
b: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
c: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
z: ValueArg, type: <auto/runtime>
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [i, j, k_outer, k_inner] : 0 <= i < Nx and 0 <= j < Ny and k_inner >= 0 and -32k_outer <= k_inner <= 31 and k_inner < Nz - 32k_outer }
---------------------------------------------------------------------------
INAME IMPLEMENT

The above splits the loop over the "iname" `k` into (a yet-undetermined number of) blocks of 32 threads each.

The "iname" (index name) `k` is gone, repalced by the combination `k_inner + k_outer * 32`.
Observe also that the "implementation" of these new inames has been tagged to map to axes of global and local parallelization (as we specified).
If we run the kernel now (enabling `write_cl` again), we see that the sequential loop over `k` is gone, and the indexing of `k` has been replaced by `32 * gid(0) + lid(0)`.

In [18]:
knl = lp.set_options(knl, write_cl=True)
evt, _ = knl(queue, a=a, b=b, c=c, z=z)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(32, 1, 1))) loopy_kernel(int const Nx, int const Ny, int const Nz, __global double *__restrict__ a, __global double const *__restrict__ b, __global double const *__restrict__ c, double const z)
{
  if (-1 + Nx >= 0 && -1 + Ny >= 0 && -1 + -32 * gid(0) + -1 * lid(0) + Nz >= 0)
    for (int j = 0; j <= -1 + Ny; ++j)
      for (int i = 0; i <= -1 + Nx; ++i)
        a[Nz * Ny * i + Nz * j + 32 * gid(0) + lid(0)] = c[Nz * Ny * i + Nz * j + 32 * gid(0) + lid(0)] * b[Nz * Ny * i + Nz * j + 32 * gid(0) + lid(0)] * b[Nz * Ny * i + Nz * j + 32 * gid(0) + lid(0)] + z;
}



Note that no loops over `k_inner` nor `k_outer` appear. They have been mapped to the "hardware" axes of parallelization: the kernel implicitly runs over a bunch of work groups (one for each value of `gid(0)`), each with 32 work items (each with their own index `lid(0)`).

And the result is still correct!

In [19]:
difference = a - a_true
np.max(difference.get())

8.881784197001252e-16

We can achieve more parallelism by "tagging" `j` and `i` as, say, global indices 1 and 2.

In [20]:
knl = lp.tag_inames(knl, {'j': 'g.1', 'i': 'g.2'})

In [21]:
knl = lp.set_options(knl, write_cl=True)
evt, _ = knl(queue, a=a, b=b, c=c, z=z)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(32, 1, 1))) loopy_kernel(int const Nx, int const Ny, int const Nz, __global double *__restrict__ a, __global double const *__restrict__ b, __global double const *__restrict__ c, double const z)
{
  if (-1 + -32 * gid(0) + -1 * lid(0) + Nz >= 0)
    a[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] = c[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] * b[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] * b[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] + z;
}



In [22]:
difference = a - a_true
np.max(difference.get())

8.881784197001252e-16

Observe the pesky `if` statement, which is ensuring that no out-of-bounds array elements are accessed by the replacement of `k` with `32 * gid(0) + lid(0)`.
If we are *sure* that this won't happen (namely, that `Nz` is divisble by 32), we can add this as an assumption:

In [23]:
knl = lp.assume(knl, 'Nz mod 32 = 0')

In [24]:
knl = lp.set_options(knl, write_cl=True)
evt, (x,) = knl(queue, a=a, b=b, c=c, z=z)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(32, 1, 1))) loopy_kernel(int const Nx, int const Ny, int const Nz, __global double *__restrict__ a, __global double const *__restrict__ b, __global double const *__restrict__ c, double const z)
{
  a[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] = c[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] * b[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] * b[Nz * Ny * gid(2) + Nz * gid(1) + 32 * gid(0) + lid(0)] + z;
}



In [25]:
difference = a - a_true
np.max(difference.get())

8.881784197001252e-16

### Symbolic representation of code

First, let's inspect the actual instruction `knl` is executing as represented by `loopy` kernel objects:

In [26]:
knl.instructions

[Assignment(assignee=Subscript(Variable('a'), (Variable('i'), Variable('j'), Sum((Variable('k_inner'), Product((Variable('k_outer'), 32)))))), predicates=frozenset(), tags=frozenset(), no_sync_with=frozenset(), boostable_into=None, boostable=None, id='insn', atomicity=(), depends_on_is_final=False, within_inames=frozenset({'i', 'k_outer', 'k_inner', 'j'}), depends_on=frozenset(), expression=Sum((Product((Power(Subscript(..., (..., ..., ...)), 2), Subscript(Variable('c'), (Variable('i'), Variable('j'), Sum((..., ...)))))), Variable('z'))), within_inames_is_final=False, conflicts_with_groups=frozenset(), temp_var_type=Optional(), priority=0, groups=frozenset())]

This has quite a lot of details - but we're interested in the "assignee" and the "expression" of the first (and only) instruction:

In [27]:
knl.instructions[0].assignee

Subscript(Variable('a'), (Variable('i'), Variable('j'), Sum((Variable('k_inner'), Product((Variable('k_outer'), 32))))))

In [28]:
knl.instructions[0].expression

Sum((Product((Power(Subscript(..., (..., ..., ...)), 2), Subscript(Variable('c'), (Variable('i'), Variable('j'), Sum((..., ...)))))), Variable('z')))

This is an *expression tree*. We can actually see what's going on if we `print(assignee, '=', expression)`:

In [29]:
print(knl.instructions[0].assignee, '=', knl.instructions[0].expression)

a[i, j, k_inner + k_outer*32] = b[i, j, k_inner + k_outer*32]**2*c[i, j, k_inner + k_outer*32] + z


This is exactly the statement that appears if we print `knl` itself:

In [30]:
print(knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: <auto/runtime>
Ny: ValueArg, type: <auto/runtime>
Nz: ValueArg, type: <auto/runtime>
a: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
b: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
c: type: <auto/runtime>, shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1) aspace: global
z: ValueArg, type: <auto/runtime>
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [i, j, k_outer, k_inner] : 0 <= i < Nx and 0 <= j < Ny and k_inner >= 0 and -32k_outer <= k_inner <= 31 and k_inner < Nz - 32k_outer }
---------------------------------------------------------------------------
INAME IMPLEMENT

Let's reproduce this instruction using `pymbolic`. First we need to import all the "primitive" objects:

In [31]:
import pymbolic.primitives as p

Now create some named "Variables" (prepending their name with an underscore so they don't overwrite our `pyopencl` arrays):

In [32]:
_a = p.Variable('a')
_b = p.Variable('b')
_c = p.Variable('c')
_z = p.Variable('z')

We also need some index variables:

In [33]:
i = p.Variable('i')
j = p.Variable('j')
k = p.Variable('k')

If we index (or "subscript") a `Variable`, we get a `Subscript` object:

In [34]:
_a[i, j, k]

Subscript(Variable('a'), (Variable('i'), Variable('j'), Variable('k')))

This matches the `assignee` of the instruction above. Let's try the `expression`:

In [35]:
_b[i, j, k]**2 * _c[i, j, k] + _z

Sum((Product((Power(Subscript(..., (..., ..., ...)), 2), Subscript(Variable('c'), (Variable('i'), Variable('j'), Variable('k'))))), Variable('z')))

In [36]:
print(_b[i, j, k]**2 * _c[i, j, k] + _z)

b[i, j, k]**2*c[i, j, k] + z


Looks good.

The lesson here is that `pymbolic` provides a symbolic way to generate code.
Rather than inputting strings of instructions to `loopy.make_kernel` (which, as we saw above, are parsed to `pymbolic` expressions by `loopy` behind the scenes!), we can work with the symbolic code directly.
This unlocks a lot of potential to actually use python as a scripting language to generate the code (which `loopy` uses to subsequently generate OpenCL code).
`pymbolic` can be thought of as a very simple computer algebra system (it can take derivatives, for instance), but geared toward manipulating and generating code.

## 3. `pystella.ElementWiseMap`

Let's see how `pystella` provides a simpler interface to `loopy` to turn `pymbolic` expressions into kernels.
Instructions are represented by lists of pairs of assignees (the left-hand side) and expressions (the right-hand side).
These can be input as dictionaries (with keys and values corresponding to assignees and expressions, respectively) or lists of tuples.

First, we'll recreate our same kernel again.

In [37]:
import pystella as ps

In [38]:
map_instructions = {
    _a[i, j, k]: _b[i, j, k]**2 * _c[i, j, k] + _z
}

In [39]:
ewmap = ps.ElementWiseMap(map_instructions, dtype='float64', halo_shape=0)

In [40]:
print(ewmap.knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: np:dtype('int64')
Ny: ValueArg, type: np:dtype('int64')
Nz: ValueArg, type: np:dtype('int64')
a: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
b: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
c: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
z: ValueArg, type: np:dtype('float64')
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [k_outer, k_inner, j_outer, j_inner, i_outer, i_inner] : i_inner = 0 an

It's the same kernel! Already parallelized---`ElementWiseMap` implements a default parallelization that works well for these types of operations.

Let's check the results:

In [41]:
evt, _ = ewmap(queue, a=a, b=b, c=c, z=z)

In [42]:
difference = a - a_true
np.max(difference.get())

8.881784197001252e-16

## 4. Using `pystella.Field`'s as input to `pystella.ElementWiseMap`

`pystella.Field`'s can make our life even easier.
Constantly indexing with `[i, j, k]` can get pretty annoying, and can be automate with `pymbolic`'s mapping methods.

In [43]:
_a = ps.Field('a')
_b = ps.Field('b')
_c = ps.Field('c')
_z = p.Variable('z')

map_instructions = [
    (_a, _b**2 * _c + _z)
]

ewmap = ps.ElementWiseMap(map_instructions, dtype='float64', halo_shape=0)

In [44]:
print(ewmap.knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: np:dtype('int64')
Ny: ValueArg, type: np:dtype('int64')
Nz: ValueArg, type: np:dtype('int64')
a: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
b: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
c: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
z: ValueArg, type: np:dtype('float64')
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [k_outer, k_inner, j_outer, j_inner, i_outer, i_inner] : i_inner = 0 an

We made no mention of indices or subscripts, yet the kernels are identical.

In [45]:
evt, _ = ewmap(queue, a=a, b=b, c=c, z=z)

In [46]:
difference = a - a_true
np.max(difference.get())

8.881784197001252e-16

To further illustrate why `pystella.Field`'s are useful, consider the (extremely common) case where arrays are padded in each direction.
This is implemented by passing a value for `offset`.

In [47]:
_a = ps.Field('a', offset='h')
_b = ps.Field('b', offset='h')
_c = ps.Field('c', offset='h')
_z = p.Variable('z')

map_instructions = {
    _a: _b**2 * _c + _z
}

ewmap = ps.ElementWiseMap(map_instructions, dtype='float64', halo_shape=1)

In [48]:
print(ewmap.knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: np:dtype('int64')
Ny: ValueArg, type: np:dtype('int64')
Nz: ValueArg, type: np:dtype('int64')
a: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
b: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
c: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
z: ValueArg, type: np:dtype('float64')
---------------------------------------------------------------------------
DOMAINS:
[Nx, 

This would get quite cumbersome to type out manually, and it's easy to forget which arrays should be padded.
From experience, it can be difficult to see errors in array indexing.

### Indexer

Behind the scenes, `ElementWiseMap` is calling `index_fields`:

In [49]:
_f = ps.Field('f', offset=('h', 3, 2))
print(_f)
print(ps.index_fields(_f))

f
f[i + h, j + 3, k + 2]


### Exercise: computing spatial gradients with `pystella.Field` and the `pystella.Stencil` kernel generator

`Field`'s indices may be shifted by `pystella.field.shift_fields`, which does what it sounds like:

In [50]:
from pystella.field import shift_fields
print(ps.index_fields(shift_fields(_a, (1, 0, 0))))

a[i + h + 1, j + h, k + h]


For this type of kernel, `pystella.Stencil` provides good parallelization (by allowing arrays to be *prefetched* into so-called "shared" memory).

In [51]:
f = ps.Field('f', offset=1)

dfdx = ps.Field('dfdx', offset=0)
dfdy = ps.Field('dfdy', offset=0)
dfdz = ps.Field('dfdz', offset=0)

dx = p.Variable('dx')

Fill in `map_instructions` below to compute the second-order centered-difference approximation to the gradient of `f`:

In [52]:
map_instructions = {
    dfdx: (shift_fields(f, (1, 0, 0)) - shift_fields(f, (-1, 0, 0))) / 2 / dx,
    dfdy: (shift_fields(f, (0, 1, 0)) - shift_fields(f, (0, -1, 0))) / 2 / dx,
    dfdz: (shift_fields(f, (0, 0, 1)) - shift_fields(f, (0, 0, -1))) / 2 / dx,
}

In [53]:
stencil = ps.Stencil(map_instructions, prefetch_args=['f'], halo_shape=1, dtype='float64')

In [54]:
print(stencil.knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: np:dtype('int64')
Ny: ValueArg, type: np:dtype('int64')
Nz: ValueArg, type: np:dtype('int64')
dfdx: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
dfdy: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
dfdz: type: np:dtype('float64'), shape: (Nx, Ny, Nz), dim_tags: (N2:stride:Nz*Ny, N1:stride:Nz, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
dx: ValueArg, type: np:dtype('float64')
f: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.

We can also do something more complicated by inputting `tmp_instructions`, which computes temporary values (that don't get stored in global arrays) before executing the assignments specified by `map_instructions`:

In [55]:
f = ps.Field('f', offset=1)
g = ps.Field('g', offset=1)

In [56]:
tmp = p.Variable('tmp')

In [57]:
tmp_instructions = {}
for i in range(3):
    shift = [0, 0, 0]
    shift[i] = 1
    expr = shift_fields(f, tuple(shift))
    shift[i] = - 1
    expr += shift_fields(f, tuple(shift))
    tmp_instructions[tmp[i]] = expr

Let's check what we just did:

In [58]:
for key, value in tmp_instructions.items():
    print(key, '=', ps.index_fields(value))

tmp[0] = f[i + 2, j + 1, k + 1] + f[i, j + 1, k + 1]
tmp[1] = f[i + 1, j + 2, k + 1] + f[i + 1, j, k + 1]
tmp[2] = f[i + 1, j + 1, k + 2] + f[i + 1, j + 1, k]


In [59]:
map_instructions = {
    g: tmp[0] * tmp[1] * tmp[2]
}

In [60]:
stencil = ps.Stencil(map_instructions, tmp_instructions=tmp_instructions, prefetch_args=['f'], halo_shape=1, dtype='float64')

print(stencil.knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
Nx: ValueArg, type: np:dtype('int64')
Ny: ValueArg, type: np:dtype('int64')
Nz: ValueArg, type: np:dtype('int64')
f: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
g: type: np:dtype('float64'), shape: (Nx + 2, Ny + 2, Nz + 2), dim_tags: (N2:stride:(Nz + 2)*(Ny + 2), N1:stride:Nz + 2, N0:stride:1), offset: <class 'loopy.kernel.data.auto'> aspace: global
---------------------------------------------------------------------------
DOMAINS:
[Nx, Ny, Nz] -> { [k_outer, k_inner, j_outer, j_inner, i_outer, i_inner, f_dim_0, f_dim_1, f_dim_2] : k_outer >= 0 and 0 <= k_inner <= 7 and k_inner < Nz - 8k_outer and j_outer >= 0 and 0 <= j_inner <= 7 and j_inner < Ny - 8j_outer and